# Fase 1: Análisis Exploratorio de Datos (EDA)

**Proyecto:** SIP Dynamic Pricing  
**Fecha:** 2026-02-19  
**Autores:** Santiago Lanz, Diego Blanco

## Objetivos
1. Entender la distribución temporal de ventas (estacionalidad)
2. Analizar el comportamiento por sucursal
3. Identificar productos clave (top volumen/ingreso)
4. Explorar la relación precio-demanda
5. Evaluar el efecto de las promociones

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

# Paths
DATA_RAW = Path('../data/raw')

print("✓ Librerías cargadas")

## 1. Carga de Datos

In [ ]:
# Cargar CompraVenta
df_cv = pd.read_parquet(DATA_RAW / 'compraventa_raw.parquet')
print(f"CompraVenta: {len(df_cv):,} registros")
print(f"Columnas: {df_cv.columns.tolist()}")

In [ ]:
# Cargar Promociones
df_promo = pd.read_parquet(DATA_RAW / 'promociones_raw.parquet')
print(f"Promociones: {len(df_promo):,} registros")
print(f"Columnas: {df_promo.columns.tolist()}")

In [ ]:
# Cargar Ajustes
df_ajustes = pd.read_parquet(DATA_RAW / 'ajustes_raw.parquet')
print(f"Ajustes: {len(df_ajustes):,} registros")
print(f"Columnas: {df_ajustes.columns.tolist()}")

In [ ]:
# Vista previa de CompraVenta
df_cv.head()

In [ ]:
# Info del dataset
df_cv.info()

In [ ]:
# Estadísticas descriptivas
df_cv.describe()

## 2. Limpieza y Preparación Inicial

In [ ]:
# Verificar valores nulos
null_counts = df_cv.isnull().sum()
null_pct = (null_counts / len(df_cv) * 100).round(2)
pd.DataFrame({'Nulos': null_counts, '%': null_pct}).query('Nulos > 0')

In [ ]:
# Calcular campos derivados
df_cv['Precio_Unitario'] = np.where(
    df_cv['Unidades_Venta_Cantidad'] > 0,
    df_cv['Precio_Venta_Total'] / df_cv['Unidades_Venta_Cantidad'],
    0
)

df_cv['Margen_Bruto'] = df_cv['Precio_Venta_Total'] - df_cv['Costo_Venta_Total']

df_cv['Margen_Porcentaje'] = np.where(
    df_cv['Precio_Venta_Total'] > 0,
    (df_cv['Margen_Bruto'] / df_cv['Precio_Venta_Total'] * 100).round(2),
    0
)

print("✓ Campos derivados calculados: Precio_Unitario, Margen_Bruto, Margen_Porcentaje")

In [ ]:
# Verificar rango de fechas
print(f"Rango de fechas: {df_cv['Fecha'].min()} a {df_cv['Fecha'].max()}")
print(f"Días únicos: {df_cv['Fecha'].nunique()}")

## 3. Análisis Temporal

In [ ]:
# Ventas diarias agregadas
ventas_diarias = df_cv.groupby('Fecha').agg({
    'Unidades_Venta_Cantidad': 'sum',
    'Precio_Venta_Total': 'sum',
    'Margen_Bruto': 'sum'
}).reset_index()

ventas_diarias.columns = ['Fecha', 'Unidades', 'Ingresos', 'Margen']

fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)

axes[0].plot(ventas_diarias['Fecha'], ventas_diarias['Unidades'], alpha=0.7)
axes[0].set_ylabel('Unidades Vendidas')
axes[0].set_title('Ventas Diarias - Unidades')

axes[1].plot(ventas_diarias['Fecha'], ventas_diarias['Ingresos'], color='green', alpha=0.7)
axes[1].set_ylabel('Ingresos ($)')
axes[1].set_title('Ventas Diarias - Ingresos')

axes[2].plot(ventas_diarias['Fecha'], ventas_diarias['Margen'], color='orange', alpha=0.7)
axes[2].set_ylabel('Margen ($)')
axes[2].set_title('Ventas Diarias - Margen Bruto')
axes[2].set_xlabel('Fecha')

plt.tight_layout()
plt.savefig('../reports/ventas_diarias.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Ventas por día de la semana
df_cv['DiaSemana'] = df_cv['Fecha'].dt.dayofweek
df_cv['NombreDia'] = df_cv['Fecha'].dt.day_name()

ventas_dia_semana = df_cv.groupby('DiaSemana').agg({
    'Unidades_Venta_Cantidad': 'sum',
    'Precio_Venta_Total': 'sum'
}).reset_index()

dias = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo']
ventas_dia_semana['Dia'] = dias

fig, ax = plt.subplots(figsize=(10, 5))
bars = ax.bar(ventas_dia_semana['Dia'], ventas_dia_semana['Precio_Venta_Total'] / 1e6)
ax.set_ylabel('Ingresos (Millones $)')
ax.set_title('Distribución de Ventas por Día de la Semana')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('../reports/ventas_dia_semana.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Ventas por mes
df_cv['AñoMes'] = df_cv['Fecha'].dt.to_period('M')

ventas_mes = df_cv.groupby('AñoMes').agg({
    'Unidades_Venta_Cantidad': 'sum',
    'Precio_Venta_Total': 'sum'
}).reset_index()

ventas_mes['AñoMes'] = ventas_mes['AñoMes'].astype(str)

fig, ax = plt.subplots(figsize=(14, 5))
ax.bar(ventas_mes['AñoMes'], ventas_mes['Precio_Venta_Total'] / 1e6)
ax.set_ylabel('Ingresos (Millones $)')
ax.set_title('Evolución Mensual de Ventas')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('../reports/ventas_mensuales.png', dpi=150, bbox_inches='tight')
plt.show()

## 4. Análisis por Categoría

In [ ]:
# Ventas por Clase
ventas_clase = df_cv.groupby('Clase').agg({
    'Unidades_Venta_Cantidad': 'sum',
    'Precio_Venta_Total': 'sum',
    'Margen_Bruto': 'sum',
    'Codigo_Interno': 'nunique'
}).reset_index()

ventas_clase.columns = ['Clase', 'Unidades', 'Ingresos', 'Margen', 'Productos']
ventas_clase['Margen_%'] = (ventas_clase['Margen'] / ventas_clase['Ingresos'] * 100).round(2)
ventas_clase = ventas_clase.sort_values('Ingresos', ascending=False)

ventas_clase

In [ ]:
# Gráfico de participación por clase
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Pie de ingresos
axes[0].pie(ventas_clase['Ingresos'], labels=ventas_clase['Clase'], autopct='%1.1f%%', startangle=90)
axes[0].set_title('Participación en Ingresos por Clase')

# Bar de margen %
colors = ['#2ecc71' if x > 20 else '#e74c3c' for x in ventas_clase['Margen_%']]
axes[1].barh(ventas_clase['Clase'], ventas_clase['Margen_%'], color=colors)
axes[1].set_xlabel('Margen Bruto (%)')
axes[1].set_title('Margen por Clase')

plt.tight_layout()
plt.savefig('../reports/analisis_clases.png', dpi=150, bbox_inches='tight')
plt.show()

## 5. Análisis por Sucursal

In [ ]:
# Ventas por sucursal
ventas_sucursal = df_cv.groupby('Sucursal').agg({
    'Unidades_Venta_Cantidad': 'sum',
    'Precio_Venta_Total': 'sum',
    'Margen_Bruto': 'sum'
}).reset_index()

ventas_sucursal.columns = ['Sucursal', 'Unidades', 'Ingresos', 'Margen']
ventas_sucursal['Margen_%'] = (ventas_sucursal['Margen'] / ventas_sucursal['Ingresos'] * 100).round(2)
ventas_sucursal = ventas_sucursal.sort_values('Ingresos', ascending=False)

ventas_sucursal

In [ ]:
# Gráfico de sucursales
fig, ax = plt.subplots(figsize=(10, 5))
bars = ax.bar(ventas_sucursal['Sucursal'], ventas_sucursal['Ingresos'] / 1e9)
ax.set_ylabel('Ingresos (Miles de Millones $)')
ax.set_title('Ingresos por Sucursal')
ax.set_xlabel('Sucursal')
plt.tight_layout()
plt.savefig('../reports/ventas_sucursal.png', dpi=150, bbox_inches='tight')
plt.show()

## 6. Top Productos

In [ ]:
# Top 20 productos por ingresos
top_productos = df_cv.groupby(['Codigo_Interno', 'Descripcion', 'Clase']).agg({
    'Unidades_Venta_Cantidad': 'sum',
    'Precio_Venta_Total': 'sum',
    'Margen_Bruto': 'sum'
}).reset_index()

top_productos.columns = ['Codigo', 'Descripcion', 'Clase', 'Unidades', 'Ingresos', 'Margen']
top_productos['Margen_%'] = (top_productos['Margen'] / top_productos['Ingresos'] * 100).round(2)

# Top por ingresos
top_ingresos = top_productos.nlargest(20, 'Ingresos')
top_ingresos

In [ ]:
# Top por volumen (unidades)
top_volumen = top_productos.nlargest(20, 'Unidades')
top_volumen

In [ ]:
# Gráfico Top 15 productos
fig, ax = plt.subplots(figsize=(12, 8))
top15 = top_ingresos.head(15)
y_pos = range(len(top15))
ax.barh(y_pos, top15['Ingresos'] / 1e6)
ax.set_yticks(y_pos)
ax.set_yticklabels(top15['Descripcion'].str[:40])
ax.invert_yaxis()
ax.set_xlabel('Ingresos (Millones $)')
ax.set_title('Top 15 Productos por Ingresos')
plt.tight_layout()
plt.savefig('../reports/top_productos.png', dpi=150, bbox_inches='tight')
plt.show()

## 7. Análisis de Precios

In [ ]:
# Distribución de precios unitarios (filtrar outliers)
precios_validos = df_cv[df_cv['Precio_Unitario'] > 0]['Precio_Unitario']

# Percentiles
print("Distribución de Precios Unitarios:")
print(precios_validos.describe(percentiles=[.25, .5, .75, .90, .95, .99]))

In [ ]:
# Distribución de precios por clase
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for i, clase in enumerate(df_cv['Clase'].unique()):
    data = df_cv[(df_cv['Clase'] == clase) & (df_cv['Precio_Unitario'] > 0) & (df_cv['Precio_Unitario'] < df_cv['Precio_Unitario'].quantile(0.99))]
    axes[i].hist(data['Precio_Unitario'], bins=50, edgecolor='black', alpha=0.7)
    axes[i].set_title(f'Distribución Precios - {clase}')
    axes[i].set_xlabel('Precio Unitario ($)')
    axes[i].set_ylabel('Frecuencia')

plt.tight_layout()
plt.savefig('../reports/distribucion_precios.png', dpi=150, bbox_inches='tight')
plt.show()

## 8. Relación Precio-Demanda (Elasticidad Preliminar)

In [ ]:
# Agregar por producto-día para ver variaciones
producto_dia = df_cv.groupby(['Codigo_Interno', 'Fecha']).agg({
    'Unidades_Venta_Cantidad': 'sum',
    'Precio_Unitario': 'mean'
}).reset_index()

# Calcular correlación por producto
correlaciones = producto_dia.groupby('Codigo_Interno').apply(
    lambda x: x['Precio_Unitario'].corr(x['Unidades_Venta_Cantidad']) if len(x) > 30 else np.nan
).dropna()

print(f"Productos con suficientes datos: {len(correlaciones)}")
print(f"\nCorrelación Precio-Demanda:")
print(f"  Media: {correlaciones.mean():.3f}")
print(f"  Mediana: {correlaciones.median():.3f}")
print(f"  % con correlación negativa: {(correlaciones < 0).mean()*100:.1f}%")

In [ ]:
# Histograma de correlaciones
fig, ax = plt.subplots(figsize=(10, 5))
ax.hist(correlaciones, bins=50, edgecolor='black', alpha=0.7)
ax.axvline(x=0, color='red', linestyle='--', label='Sin correlación')
ax.axvline(x=correlaciones.median(), color='green', linestyle='--', label=f'Mediana: {correlaciones.median():.2f}')
ax.set_xlabel('Correlación Precio-Demanda')
ax.set_ylabel('Número de Productos')
ax.set_title('Distribución de Correlación Precio-Demanda por Producto')
ax.legend()
plt.tight_layout()
plt.savefig('../reports/correlacion_precio_demanda.png', dpi=150, bbox_inches='tight')
plt.show()

## 9. Análisis de Promociones

In [ ]:
# Resumen de promociones
print(f"Total promociones: {df_promo['Cod_Promocion'].nunique():,}")
print(f"Productos con promoción: {df_promo['Cod_Producto'].nunique():,}")
print(f"\nDistribución por tipo:")
print(df_promo['Tipo_Promocion'].value_counts())

In [ ]:
# Tipos de promoción
tipo_promo_desc = {
    1: 'Precio Oferta',
    2: '% Descuento',
    3: 'Descuento Fijo',
    4: 'M×N Gratis',
    5: 'M×N Precio Oferta',
    6: 'M×N % Descuento',
    7: 'M×N Monto Fijo',
    8: 'Premio M×N',
    9: 'Premio Precio Oferta',
    10: 'Premio % Descuento',
    11: 'Premio Monto Descuento'
}

promo_counts = df_promo['Tipo_Promocion'].value_counts()
promo_df = pd.DataFrame({'Tipo': promo_counts.index, 'Cantidad': promo_counts.values})
promo_df['Descripcion'] = promo_df['Tipo'].map(tipo_promo_desc)
promo_df['%'] = (promo_df['Cantidad'] / promo_df['Cantidad'].sum() * 100).round(1)
promo_df

In [ ]:
# Gráfico de tipos de promoción
fig, ax = plt.subplots(figsize=(12, 6))
promo_df_sorted = promo_df.sort_values('Cantidad', ascending=True)
ax.barh(promo_df_sorted['Descripcion'], promo_df_sorted['Cantidad'])
ax.set_xlabel('Cantidad de Registros')
ax.set_title('Distribución de Tipos de Promoción')
plt.tight_layout()
plt.savefig('../reports/tipos_promocion.png', dpi=150, bbox_inches='tight')
plt.show()

## 10. Análisis de Ajustes de Inventario

In [ ]:
# Resumen de ajustes
print(f"Total ajustes: {len(df_ajustes):,}")
print(f"Productos con ajustes: {df_ajustes['Codigo_Producto'].nunique():,}")
print(f"\nDistribución por tipo de documento:")
print(df_ajustes['Tipo_Documento'].value_counts())

In [ ]:
# Ajustes por código de razón (si tiene datos)
if 'Codigo_Razon' in df_ajustes.columns:
    print("\nDistribución por Código de Razón:")
    print(df_ajustes['Codigo_Razon'].value_counts().head(20))

In [ ]:
# Productos con más ajustes (posibles productos problemáticos)
ajustes_por_producto = df_ajustes.groupby('Codigo_Producto').agg({
    'Cantidad': ['count', 'sum']
}).reset_index()
ajustes_por_producto.columns = ['Codigo_Producto', 'Num_Ajustes', 'Cantidad_Total']
ajustes_por_producto = ajustes_por_producto.sort_values('Num_Ajustes', ascending=False)

print("Top 20 productos con más ajustes:")
ajustes_por_producto.head(20)

## 11. Resumen Ejecutivo

In [ ]:
# Métricas clave
print("="*60)
print("RESUMEN EJECUTIVO - EDA")
print("="*60)

print(f"\n📊 DATOS ANALIZADOS")
print(f"   Registros CompraVenta: {len(df_cv):,}")
print(f"   Período: {df_cv['Fecha'].min().strftime('%Y-%m-%d')} a {df_cv['Fecha'].max().strftime('%Y-%m-%d')}")
print(f"   Sucursales: {df_cv['Sucursal'].nunique()}")
print(f"   Productos únicos: {df_cv['Codigo_Interno'].nunique():,}")

print(f"\n💰 MÉTRICAS FINANCIERAS")
print(f"   Ingresos totales: ${df_cv['Precio_Venta_Total'].sum():,.2f}")
print(f"   Margen bruto total: ${df_cv['Margen_Bruto'].sum():,.2f}")
print(f"   Margen bruto promedio: {(df_cv['Margen_Bruto'].sum() / df_cv['Precio_Venta_Total'].sum() * 100):.1f}%")

print(f"\n📈 PATRONES IDENTIFICADOS")
print(f"   Correlación precio-demanda negativa en {(correlaciones < 0).mean()*100:.1f}% de productos")
print(f"   {len(df_promo):,} registros de promociones disponibles")
print(f"   {len(df_ajustes):,} registros de ajustes de inventario")

print("\n" + "="*60)

## Próximos Pasos

1. **Fase 2:** Diseño de arquitectura del modelo de datos
2. **Fase 3:** Pipeline ETL - Integración de CompraVenta + Promociones
3. **Fase 4:** Feature Engineering detallado